# Regression Analysis on Evictions, Poverty, and Race
In the "enhancing-statistical-rigor" notebook it became clear that the EL mislabeled the flag columns:
- imputed -- was accidentally labeled --> low-flag
- subbed -- was accidentally labeled --> imputed
- low-flag -- was accidentally labeled --> subbed.

So, I'll fix the labels and then compare regression analyses on:
- all the data
- filtered data (i.e. drop the data points that have been flagged as low)

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
import cufflinks as cf # cufflinks binds plotly to pandas dataframes in IPython notebook.
import plotly.graph_objs as go
from plotly import tools

In [2]:
sns.set(style = 'darkgrid')

## Read in the County Level Data and correct the labels

In [3]:
counties_evicts_df = pd.read_csv('data/counties.csv')
counties_evicts_df.rename(columns = {'low-flag': 'imputed',
                                    'imputed': 'subbed',
                                    'subbed': 'low-flag'}, inplace = True)
counties_evicts_df.head()

,GEOID,year,name,parent-location,population,poverty-rate,pct-renter-occupied,median-gross-rent,median-household-income,median-property-value,...,pct-multiple,pct-other,renter-occupied-households,eviction-filings,evictions,eviction-rate,eviction-filing-rate,imputed,subbed,low-flag
0,47001,2000,Anderson County,Tennessee,71330.0,13.14,27.49,450.0,35483.0,87500.0,...,1.14,0.07,8189.0,125.0,118.0,1.44,1.53,0,0,1
1,47003,2000,Bedford County,Tennessee,37586.0,13.14,26.46,488.0,36729.0,79000.0,...,0.77,0.05,3679.0,104.0,98.0,2.66,2.83,0,0,0
2,47005,2000,Benton County,Tennessee,16537.0,15.63,19.47,367.0,28679.0,67000.0,...,0.62,0.03,1336.0,17.0,17.0,1.27,1.27,0,0,0
3,47007,2000,Bledsoe County,Tennessee,12367.0,18.07,18.24,343.0,28982.0,67200.0,...,1.03,0.02,808.0,0.0,0.0,0.00,0.00,0,0,0
4,47009,2000,Blount County,Tennessee,105823.0,9.73,24.11,450.0,37862.0,103900.0,...,0.88,0.05,10286.0,210.0,201.0,1.95,2.04,0,0,1


Confirm my 'low-flag' column corroborates the criteria stated in pg. 40 of the methodology report
- *i.e. that at least 25% of the counties between 2009 - 2016 are flagged as low*
    - that's from January 1st 2009 - January 1st 2016